In [1]:
import pandas as pd
pd.options.display.float_format = '{:20.2f}'.format

# Prepare energy production dataset

We used the bavaria energy production.

In [2]:
# open the dataset
co2_emissions = pd.read_excel('input/Bavaria_produced.xlsx')

# convert the date to standard string format
co2_emissions['Datum'] = co2_emissions['Datum'].str[-4:] + '-' + co2_emissions['Datum'].str[3:5] + '-' + co2_emissions['Datum'].str[:2]

# transform the date to pandas datetime
co2_emissions['Date'] = pd.to_datetime(co2_emissions.Datum + ' ' + co2_emissions.Uhrzeit +':00') 

# calculate the total kgCO2eq per hour
co2_emissions['kgCO2eq'] = (
    (co2_emissions['Biomasse[MWh]'] * 50.4) +
    (co2_emissions['Wasserkraft[MWh]'] * 22.5) +
    (co2_emissions['Wind Offshore[MWh]'] * 0.165) +
    (co2_emissions['Wind Onshore[MWh]'] * 0.165) +
    (co2_emissions['Photovoltaik[MWh]'] * 0.00448) +
    (co2_emissions['Sonstige Erneuerbare[MWh]'] * 0.00664) +
    (co2_emissions['Kernenergie[MWh]'] * 9.37) +
    (co2_emissions['Braunkohle[MWh]'] * 1160) +
    (co2_emissions['Steinkohle[MWh]'] * 1160) +
    (co2_emissions['Erdgas[MWh]'] * 440) +
    (co2_emissions['Sonstige Konventionelle[MWh]'] * 875) +
    (co2_emissions['Pumpspeicher[MWh]'] * 958)
)


# Transform to kWh 
co2_emissions['TotalkWh'] = co2_emissions[['Biomasse[MWh]', 'Wasserkraft[MWh]',
       'Wind Offshore[MWh]', 'Wind Onshore[MWh]', 'Photovoltaik[MWh]',
       'Sonstige Erneuerbare[MWh]', 'Kernenergie[MWh]', 'Braunkohle[MWh]',
       'Steinkohle[MWh]', 'Erdgas[MWh]', 'Pumpspeicher[MWh]',
       'Sonstige Konventionelle[MWh]']].sum(1) * 1000


# kgCO2eq per kWH
co2_emissions['kgCO2eq/kWh'] = co2_emissions['kgCO2eq'] /co2_emissions['TotalkWh']


# extract and rename the relevant columns
co2_emissions = co2_emissions[
    ['Date','Rens', 'NonRens', 'TotalkWh','kgCO2eq', 'kgCO2eq/kWh']].rename(columns = {'Date':'interval'})


# Prepare mail days and mailed clients datasets

The data is store in an Excel file in which different sheets contains a date with the client ids that were mailed. The file also contains the sheet (`alle Coder`) with the ids of all the clients of the company, and that were candidates of being mailed in a mail day.

In [3]:
# read all the ids/codes
codes = set(pd.read_excel('input/mails_coders.xls', sheet_name='alle Coder', header=None)[0])
print(f"Total clients: {len(codes)}")

# read the raw data were the information about the mails was kept
dfs = pd.read_excel('input/mails_coders.xls', sheet_name=None)

# list to keep the dataframe in each
dfs_list = []

# loop through the Excel sheets
for key, df in dfs.items():
    
    # make sure that the name of the sheet contains a date in the format DDMMYYYY
    if key.isnumeric() and len(key) == 8:
        
        # transform the date to the right string format
        df['mailday'] = key[-4:] + '-' + key[2:4] + '-' + key[:2] + ' 00:00:00'
        
        # append to the list 
        dfs_list.append(df)

# create the dataframe, and rename columns
client_mails = pd.concat(dfs_list).rename(columns={
    'Coder': 'client_id',
    'mailday': '17h_day'
})

# all of the ones in this dataset were mailed
client_mails['mailed'] = True

# convert day to datetime
client_mails['17h_day'] = pd.to_datetime(client_mails['17h_day'])

# remove duplicates
client_mails = client_mails.drop_duplicates(['client_id', '17h_day'])

# create dataset that identifies maildays
maildays = client_mails[['17h_day', 'mailed']].drop_duplicates().rename(columns={'mailed': 'mailday'})


Total clients: 318


# Prepare charge events data source 
`greenweeks_charges.csv` contains the raw data obtain from the provider

## Read and clean the charge events



In [4]:
# read the file
charge_events = pd.read_csv('input/greenweeks_charges.csv')

# print raw statistics
print("Descriptive statistics of the raw data:")
print(f"Total Charges (N): {len(charge_events)}")
print(f"Total Clients: {len(charge_events.KundenID.unique())}")
print(charge_events[['Total charge amount','Total duration']].describe())

# numerate rows by using the current index
charge_events = charge_events.reset_index()

# create a column map with new column names to simplify access to them
colmap = {
    'KundenID': 'client_id',
    'index': 'charge_id',
    'EVSE ID': 'ev_id',
    'Start time': 'starttime',
    'End time': 'endtime',
    # Where does this come frome? Is it broken?
    # 'Mailversand': 'mail',
    'Total duration': 'total_duration',
    'Meter start': 'meter_start', 
    'Meter end': 'meter_end', 
    'Total charge amount': 'total_charge',
    'Charging station identifier': 'station_id',
    'Charging station': 'station',
    'Charging station serialnumber': 'station_sn',
    'Charging station street': 'station_st',
}

# only consider the rename columns
charge_events = charge_events[list(colmap.keys())]

# actually rename the columns
charge_events = charge_events.rename(columns=colmap)

# filter for low duration and low charge amount
charge_events = charge_events[~((charge_events.total_duration<1*60) | (charge_events.total_charge<50))]

# transform dates from strings to pandas datetimes
charge_events['starttime'] = pd.to_datetime(charge_events['starttime'], format="%d/%m/%Y %H:%M")

# simplify the identification of EVs
charge_events['ev_id'] = charge_events['ev_id'].str.split('*').str[-1]

# calculate the endtime using the duration
charge_events['endtime'] = charge_events['starttime'] + pd.to_timedelta(charge_events['total_duration'], unit='s')

# removing missing values
charge_events = charge_events[charge_events['endtime'].notnull()]

# transform Wh to kWh
charge_events['total_charge'] = charge_events['total_charge'] / 1000

# summary statistics after basic preparation of the data
print("Summary statistics of the clean raw data")
charge_events.describe()

Descriptive statistics of the raw data:
Total Charges (N): 270
Total Clients: 91
       Total charge amount       Total duration
count               269.00               269.00
mean               9384.35              5919.74
std                8718.97              8383.75
min                   0.00                 2.00
25%                3308.00              1689.00
50%                6510.00              4022.00
75%               13488.00              7647.00
max               38680.00             86456.00
Summary statistics of the clean raw data


,client_id,charge_id,total_duration,meter_start,meter_end,total_charge,station_id
count,247.00,247.00,247.00,247.00,247.00,247.00,247.00
mean,7153.24,135.56,6409.71,532760.36,542980.10,10.22,15931.31
std,89.01,78.27,8566.43,1016667.97,1016942.88,8.62,34140.55
min,7005.00,0.00,159.00,0.00,126.00,0.13,45.00
25%,7083.00,68.50,2186.50,0.00,5519.00,3.93,889.00
50%,7159.00,135.00,4860.00,0.00,13088.00,7.02,1102.00
75%,7238.00,202.50,8044.50,519075.00,525645.00,15.26,20681.00
max,7317.00,269.00,86456.00,4090687.00,4098083.00,38.68,142039.00


## Represent the charge event in 15 minutes intervals

For example, a charge event that occured on the 2019-03-20 that starts at 13:05 and ends at 13:40 is divided in 3 sections:

- `2019-03-20 13:05`-`2019-03-20 13:15` containing `10` minutes that belongs to the interval `2019-03-20 13:00`-`2019-03-20 13:15`
- `2019-03-20 13:15`-`2019-03-20 13:30` containing `15` minutes that belongs to the interval `2019-03-20 13:15`-`2019-03-20 13:30`
- `2019-03-20 13:30`-`2019-03-20 13:35` containing `5` minutes that belongs to the interval `2019-03-20 13:30`-`2019-03-20 13:45`

In terms of nomenclature, the above sections fullfills the following template (replacing the values by variable names)

`[start]`-`[end]` containing `[duration]` minutes that belongs to the interval `[inverval]` - `[interval + 15minutes]`

Additionally: 
- `interval_time` refers to the starting time (`13:05`) of the interval without the date
- `17h_day` corresponds to our day of interest (17h shift) according to our mail intervention. Given that we sent emails ath 16h, a day starts at 17h and ends at 17h of the next day.
- `critical` will signal if the time falls into critical time (11:00 to 15:00)


In [5]:
# Split into quarters
intervals = pd.DataFrame(((
                        e.charge_id, 
                        # 17 hour day
                        (date - pd.Timedelta(hours=17)).floor('1D'),
                        date,
                        e.starttime if date == e.starttime.floor('15min') else date,
                        e.endtime if date == e.endtime.floor('15min') else date + pd.Timedelta(minutes=15),
                        date == e.starttime.floor('15min')
                    )    
                    #iterate over thet uples
                    for e  in charge_events.itertuples()
                    # iterate for each range date
                    for date in pd.date_range(e.starttime.floor('15min'), e.endtime.floor('15min'), freq='15min')
                ), columns = ['charge_id', '17h_day','interval', 'start', 'end', 'charge_started'])

# just the time of the interval
intervals['interval_time'] = pd.to_timedelta(intervals['interval'].dt.time.astype(str))

# is critical time
intervals['critical'] = (intervals['interval_time'] >= pd.Timedelta('11:00:00')) & (intervals['interval_time'] < pd.Timedelta('15:00:00'))

# leave it as string, because pandas don't store this appropiatedly as CSV
intervals['interval_time'] = intervals['interval'].dt.time.astype(str)

# calculate the duration
intervals['duration'] = intervals['end']- intervals['start']

# duration in seconds
intervals['secs'] = intervals['duration'].dt.total_seconds()

# merge back to includes the rest of the columns of clean data, e.g. client_id, starttime, endtime
charge_events_15m = intervals.merge(charge_events, how='left', on='charge_id')

# calculate the duration weight
charge_events_15m['weight'] = charge_events_15m['duration'].dt.total_seconds() / charge_events_15m['total_duration']

# calculate the charge
charge_events_15m['charge'] = charge_events_15m['total_charge'] * charge_events_15m['weight']

# select relevan columns for the rest of the analysis
charge_clients_15m = charge_events_15m[[
    'client_id', 'charge_id', 'ev_id', 
    'station_id', 'station', 'station_sn','station_st',
    'charge_started',  'start', 'critical', 'end',  '17h_day', 
    'interval_time', 'interval', 'duration', 'secs', 'starttime', 'endtime', 
    'total_duration', 'total_charge', 'weight', 'charge']]

# Final shapes
print(intervals.shape, charge_events_15m.shape, charge_events.shape)

(2005, 10) (2005, 24) (247, 13)


## Sanity checks 

In [6]:
print("Comparing the total charges between the datasets")
if charge_events.total_charge.sum().round(8) == charge_events_15m.charge.sum().round(8):
    print('Passed. The total charge is the same.')
else:
    print(f"Total charge in source dataset: {charge_events.total_charge.sum()}")
    print(f"Total charge in the 15-minutes interval dataset: {charge_events_15m.charge.sum()}")
    raise Exception('Something went wrong creation the 15-minutes dataset. ', 
                    'The total charges should be the same, yet they are not.')


# Calculate the durations on the source dataset
charge_events_indexed = charge_events.set_index('charge_id')
totals = (charge_events_indexed['endtime'] - charge_events_indexed['starttime']).sort_index()

# Calculate the durations in the 15-minutes interval dataset
_15min_totals = charge_events_15m.groupby('charge_id')['duration'].sum().sort_index()
 
print("Comparing the duration per charge id between the datasets")
if _15min_totals.equals(totals):
    print('Passed. The durations per charge are the same.')
else:
    raise Exception('Something went wrong creation the 15-minutes dataset. ', 
                    'The durations per charge should be the same, yet they are not.')
    

Comparing the total charges between the datasets
Passed. The total charge is the same.
Comparing the duration per charge id between the datasets
Passed. The durations per charge are the same.


## Charges per client in 15 minutes intervals

Similar to the charge event in 15 minutes intervals, but the unit used is clients insteadof the events. In 5 cases, doing this per charge id would produce inconsistencies in the representation of the data, because of consecutive charges during the same 15 minutes interval on the same quarter.

In [7]:
# Copy the events dataframe
charge_clients_15m = charge_events_15m.copy()

# Two charge ids could fall into the same interval, so a new column (charge ids)
# will hold this information
charge_clients_15m['charge_ids'] = charge_clients_15m['charge_id']

# Group the data by client_id and intervat. The rest of the columns in the groupby
# operation are redundant but they are kept so that they appear in the resulting 
# dataframe (otherwise they should be merge back)
charge_clients_15m = charge_clients_15m.groupby(
    ['client_id', 'interval', '17h_day', 'interval_time', 'critical',
      'station_id', 'station', 'station_sn','station_st']).agg({
        # why max? check client_id = 7257, and interval_day = 2019-02-27
        'charge_started': 'max', 
        'charge_id': 'max',
        'charge_ids': lambda x: tuple(x),
        'ev_id': lambda x: tuple(x),
        'starttime': 'min',
        'endtime': 'max',
        'start': 'min',
        'end': 'max',
        'duration': 'sum',
        'secs': 'sum',
        'charge': 'sum'
}).reset_index()


# select relevant columns for the rest of the analysis
charge_clients_15m = charge_clients_15m[[
    'client_id', 'charge_id', 'charge_ids', 'ev_id', 'station_id', 'station_st', 
    'charge_started', 'start', 'critical', 'end', '17h_day', 
    'interval_time', 'interval', 'duration', 'secs', 'charge']]
 
# Quick sanity check
print("Comparing the total charges between the datasets")
if charge_clients_15m.charge.sum() == charge_clients_15m.charge.sum():
    print('Passed. The total charge is the same.')
else:
    print(f"Total charge in source dataset: {charge_clients_15m.total_charge.sum()}")
    print(f"Total charge in the 15-minutes interval dataset: {charge_clients_15m.charge.sum()}")
    raise Exception('Something went wrong creation the 15-minutes dataset. ', 
                    'The total charges should be the same, yet they are not.')

Comparing the total charges between the datasets
Passed. The total charge is the same.


# Summary statistics per charge event

In [8]:
# merge with the energy emissions dataset
_merged = charge_events_15m.merge(co2_emissions[['interval','kgCO2eq/kWh']], how='left', on='interval')

# calculate kgCO2eq
_merged['kgCO2eq'] = _merged['charge'] * _merged['kgCO2eq/kWh']

# merge with the client mails information to identify those who where mailed
_merged = _merged.merge(client_mails, how='left', on=['17h_day','client_id'])

# identify the maildays
_merged = _merged.merge(maildays, how='left', on='17h_day')

# missings correspond to false for mailed
_merged['mailed'] = _merged['mailed'].notnull() 

# missings correspond to false for mailday
_merged['mailday'] = _merged['mailday'].notnull() 

_merged['charges started'] = _merged['charge_started'].astype(int)

# display summary statistics
print(f'Clients charging on maildays: {len(_merged[_merged.mailday].client_id.unique())}')
print(f'Clients charging on non maildays: {len(_merged[~_merged.mailday].client_id.unique())}')
print(f'Charge events on maildays: {len(_merged[_merged.mailday].charge_id.unique())}')
print(f'Charge events on non maildays: {len(_merged[~_merged.mailday].charge_id.unique())}')

print("\nFull summary")
print(_merged.groupby(['charge_id']).agg({'charges started': 'sum', 'charge': 'sum', 'kgCO2eq': 'sum'}).describe())
print("Totals:")
print(_merged[['charges started','charge', 'kgCO2eq']].sum())

print("\nSummary of Mail Days")
print(_merged[_merged.mailday].groupby(['charge_id']).agg({'charges started': 'sum', 'charge': 'sum', 'kgCO2eq': 'sum'}).describe())
print("Totals:")
print(_merged[_merged.mailday][['charges started', 'charge', 'kgCO2eq']].sum())

print("\nFull of Non Mail Days")
print(_merged[~_merged.mailday].groupby(['charge_id']).agg({'charges started': 'sum', 'charge': 'sum', 'kgCO2eq': 'sum'}).describe())
print("Totals:")
print(_merged[~_merged.mailday][['charges started', 'charge', 'kgCO2eq']].sum())

print("\nFull summary (per day)")
print(_merged.groupby(['17h_day']).agg({'charges started': 'sum', 'charge': 'sum', 'kgCO2eq': 'sum'}).describe())
print("Totals:")
print(_merged[['charges started', 'charge', 'kgCO2eq']].sum())


Clients charging on maildays: 66
Clients charging on non maildays: 53
Charge events on maildays: 135
Charge events on non maildays: 126

Full summary
           charges started               charge              kgCO2eq
count               247.00               247.00               247.00
mean                  1.00                10.22                 1.22
std                   0.00                 8.62                 1.78
min                   1.00                 0.13                 0.02
25%                   1.00                 3.93                 0.32
50%                   1.00                 7.02                 0.66
75%                   1.00                15.26                 1.22
max                   1.00                38.68                14.18
Totals:
charges started                 247.00
charge                         2524.27
kgCO2eq                         301.98
dtype: float64

Summary of Mail Days
           charges started               charge              kgCO2e

# Create main dataframe of analysis per client

In [9]:
# merge with the energy emissions dataset
main_df = charge_clients_15m.merge(co2_emissions[['interval','kgCO2eq/kWh']], how='left', on='interval')

# calculate kgCO2eq
main_df['kgCO2eq'] = main_df['charge'] * main_df['kgCO2eq/kWh']

# merge with the client mails information to identify those who where mailed
main_df = main_df.merge(client_mails, how='left', on=['17h_day','client_id'])

# identify the maildays
main_df = main_df.merge(maildays, how='left', on='17h_day')

# missings correspond to false for mailed
main_df['mailed'] = main_df['mailed'].notnull() 

# missings correspond to false for mailday
main_df['mailday'] = main_df['mailday'].notnull() 


##   Summary statistics per charge event

In [10]:
main_df['charges started'] = main_df['charge_started'].astype(int)

print("\nFull summary")
print(main_df.groupby(['client_id']).agg({'charge_started': 'sum', 'charge': 'sum', 'kgCO2eq': 'sum'}).describe())
print("Totals:")
print(main_df[['charge_started', 'charge', 'kgCO2eq']].sum())

print("\nSummary of Mail Days")
print(main_df[main_df.mailday].groupby(['client_id']).agg({'charge_started': 'sum', 'charge': 'sum', 'kgCO2eq': 'sum'}).describe())
print("Totals:")
print(main_df[main_df.mailday][['charge_started', 'charge', 'kgCO2eq']].sum())

print("\nFull of Non Mail Days")
print(main_df[~main_df.mailday].groupby(['client_id']).agg({'charge_started': 'sum', 'charge': 'sum', 'kgCO2eq': 'sum'}).describe())
print("Totals:")
print(main_df[~main_df.mailday][['charge_started', 'charge', 'kgCO2eq']].sum())

main_df.drop(columns = ['charges started'])


Full summary
            charge_started               charge              kgCO2eq
count                90.00                90.00                90.00
mean                  2.74                28.05                 3.36
std                   2.58                41.16                 6.05
min                   1.00                 0.57                 0.04
25%                   1.00                 5.94                 0.57
50%                   2.00                17.24                 1.36
75%                   3.00                33.02                 3.08
max                  13.00               316.42                44.23
Totals:
charge_started                 247.00
charge                        2524.27
kgCO2eq                        301.98
dtype: float64

Summary of Mail Days
            charge_started               charge              kgCO2eq
count                66.00                66.00                66.00
mean                  1.95                19.77                 1.53

,client_id,charge_id,charge_ids,ev_id,station_id,station_st,charge_started,start,critical,end,17h_day,interval_time,interval,duration,secs,charge,kgCO2eq/kWh,kgCO2eq,mailed,mailday
0,7005,105,"(105,)","(E1000311,)",20818,Bahnhofsplatz 1,True,2019-03-19 15:06:00,False,2019-03-19 15:15:00,2019-03-18,15:00:00,2019-03-19 15:00:00,00:09:00,540.00,2.67,0.18,0.49,False,False
1,7005,105,"(105,)","(E1000311,)",20818,Bahnhofsplatz 1,False,2019-03-19 15:15:00,False,2019-03-19 15:16:54,2019-03-18,15:15:00,2019-03-19 15:15:00,00:01:54,114.00,0.56,0.19,0.11,False,False
2,7009,257,"(257,)","(E1000119,)",924,Hintermarkt-Parkplatz,True,2019-02-11 19:45:00,False,2019-02-11 20:00:00,2019-02-11,19:45:00,2019-02-11 19:45:00,00:15:00,900.00,0.33,0.13,0.04,False,False
3,7009,257,"(257,)","(E1000119,)",924,Hintermarkt-Parkplatz,False,2019-02-11 20:00:00,False,2019-02-11 20:15:00,2019-02-11,20:00:00,2019-02-11 20:00:00,00:15:00,900.00,0.33,0.16,0.05,False,False
4,7009,257,"(257,)","(E1000119,)",924,Hintermarkt-Parkplatz,False,2019-02-11 20:15:00,False,2019-02-11 20:30:00,2019-02-11,20:15:00,2019-02-11 20:15:00,00:15:00,900.00,0.33,0.16,0.05,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,7317,21,"(21,)","(E1000546,)",20697,Friedenstraße 2,False,2019-03-17 12:15:00,True,2019-03-17 12:30:00,2019-03-16,12:15:00,2019-03-17 12:15:00,00:15:00,900.00,1.29,0.03,0.04,True,True
1996,7317,21,"(21,)","(E1000546,)",20697,Friedenstraße 2,False,2019-03-17 12:30:00,True,2019-03-17 12:45:00,2019-03-16,12:30:00,2019-03-17 12:30:00,00:15:00,900.00,1.29,0.03,0.04,True,True
1997,7317,21,"(21,)","(E1000546,)",20697,Friedenstraße 2,False,2019-03-17 12:45:00,True,2019-03-17 13:00:00,2019-03-16,12:45:00,2019-03-17 12:45:00,00:15:00,900.00,1.29,0.03,0.04,True,True
1998,7317,21,"(21,)","(E1000546,)",20697,Friedenstraße 2,False,2019-03-17 13:00:00,True,2019-03-17 13:15:00,2019-03-16,13:00:00,2019-03-17 13:00:00,00:15:00,900.00,1.29,0.03,0.04,True,True


# Generate dataset version for visualization and statistical analysis

## Compact dataset with critical vs non critical information

In [11]:
# columns of interests 
coi = ['client_id', 'charge_id', 'station_id', 'critical', 'charge_started', 
       '17h_day', 'interval_time', 'charge', 'kgCO2eq','secs']

# aggregate according to clients, days and critical
critical_df = main_df[coi].groupby(['client_id', '17h_day', 'critical']).agg({
    'charge_id': lambda x: set(x),
    'station_id': lambda x: set(x),
    'charge_started': 'any',
    'charge': 'sum',
    'kgCO2eq': 'sum',
    'secs': 'sum'
})

# store as a pickle to keep pandas data types
critical_df.to_pickle('output/critical.pickle')

# alternatively, export it as CSV
#critical_df.to_csv('output/critical.csv', index=False)

## Long format dataset containing critical times when no charges occured

This representation of the data accounts for the times in which the clients did not charge (the zeros rows)

In [12]:
# create a skeleton for the long format representation
ldf = pd.DataFrame(((x, y, z) 
                    for x in codes 
                    for y in pd.date_range(main_df['17h_day'].min(), main_df['17h_day'].max(), freq='1D') 
                    for z in (False, True)),
                   columns=('client_id', '17h_day', 'critical'))
print(ldf.shape)

# tag maildays in the dataset
ldf = ldf.merge(main_df[['17h_day', 'mailday']].drop_duplicates(), how='left', on='17h_day')
print(ldf.shape)

# merge with the critical dataframe
ldf = ldf.merge(critical_df.reset_index(), how='left', on=(
    'client_id', '17h_day', 'critical'), indicator=True)
print(ldf.shape)

ldf = ldf.merge(client_mails, how='left', on=['17h_day', 'client_id'])
print(ldf.shape)
    
#ldf = ldf.merge(critical_df.reset_index(), how='left', on=('client_id', '17h_day', 'critical'), indicator=True)
ldf[['mailed', 'mailday', 'charge_started']] = ldf[['mailed', 'mailday', 'charge_started']].fillna(False).astype(bool)

# assign right types to columns
ldf['charge'] = ldf['charge'].fillna(0)

#ldf[['weight', 'charge', 'kgCO2eq']] = ldf[['weight', 'charge', 'kgCO2eq']].fillna(0)
# weekends go from Friday 17h to Sunday 17h, after the 17h shift, this means 
# Fridays and Saturdays corresponds to weekends
ldf['is_weekday'] = ~((ldf['17h_day'].dt.weekday == 4) | 
                     (ldf['17h_day'].dt.weekday == 5) | 
                     (ldf['17h_day'] == '2019-03-03'))


# save this as a Python pickle to not loss data types
ldf.to_pickle('output/critical_long_format.pickle')

# or export as CSV
# ldf.to_csv('output/50-critical_long_format.csv', index=False)
ldf.to_csv('output/critical_long_format.csv', index=False)

# Add a dummy to all conditions statistical analysis purposes
dummies = []
for x,g in ldf.groupby(['is_weekday', 'mailday', 'critical', 'mailed']):
    dummies.append((*x,1,1/3600,1,1))
dfdummy = pd.DataFrame(dummies, columns=[
    'is_weekday', 'mailday', 'critical', 'mailed', 
    'charge', 'hours', 'kgCO2eq', 'charge_started' ])
ldf_dummy = pd.concat([ldf,dfdummy])

# code booleans as binaries
ldf_dummy[ldf_dummy.select_dtypes('boolean').columns] = ldf_dummy.select_dtypes('boolean').astype(int)
ldf[ldf.select_dtypes('boolean').columns] = ldf.select_dtypes('boolean').astype(int)

# code missing charges as 0s
ldf_dummy['charge'] = ldf_dummy['charge'].fillna(0)
ldf['charge'] = ldf['charge'].fillna(0)

# save to csv (so it is compatible with R)
ldf_dummy.to_csv('output/critical_long_format_with_dummy.csv', index=False)
ldf.to_csv('output/critical_long_format.csv', index=False)

(26712, 3)
(26712, 4)
(26712, 11)
(26712, 12)


## Long format dataset containing information about charges in 15 minutes interval regardles if there where or not charges

In [13]:
# create a skeleton for the 15minutes long format representation
uldf = pd.DataFrame(((client, interval) 
                    for client in codes 
                    for interval in pd.date_range(main_df.interval.min(), main_df.interval.max(), freq='15min')),
                   columns=('client_id', 'interval'))

# calculate the day
#uldf['interval_day'] = pd.to_datetime(uldf.interval.dt.date)
uldf['17h_day'] = pd.to_datetime((uldf.interval - pd.Timedelta(hours=17)).dt.date)

print(uldf.shape)

# # merge with maildays
uldf = uldf.merge(maildays, how='left', on='17h_day')
print(uldf.shape)

# merge with the client mails
uldf = uldf.merge(client_mails, how='left', on=['17h_day', 'client_id'])
print(uldf.shape)
      
# merge with charges of clients
uldf = uldf.merge(main_df[['client_id', 'interval', 'charge', 'kgCO2eq']], 
                  how='left', on=['client_id', 'interval'], indicator=True)
print(uldf.shape)
      
# # mark weekdays
uldf['is_weekday'] = ~((uldf['17h_day'].dt.weekday == 4) | 
                      (uldf['17h_day'].dt.weekday == 5) | 
                      (uldf['17h_day'] == '2019-03-03'))

# assign right types to columns
uldf['mailed'] = uldf['mailed'].fillna(False).astype(bool)
uldf['mailday'] = uldf['mailday'].fillna(False).astype(bool)
uldf['interval_time'] = uldf['interval'].dt.time

# fill missings with zeros
uldf['charge'] = uldf['charge'].fillna(0)
uldf['kgCO2eq'] = uldf['kgCO2eq'].fillna(0)
                                         
# flag to indicate if there is charging happening
uldf['charging'] = uldf['charge'] > 0

# Adding co2/kwh
uldf = uldf.merge(co2_emissions[['interval','kgCO2eq/kWh']], how='left', on='interval')

# Store as pickle
uldf.to_pickle('output/critical_15min_long_format.pickle')

(1239564, 3)
(1239564, 4)
(1239564, 5)
(1239564, 8)
